In [1]:
import os
import pickle
import time

import cv2
import argparse
import numpy as np
from keras.applications.mobilenet import preprocess_input
from model_util import DeepModel
from multi import ImageSimilarity

In [2]:
class ImageClassifier:
    def __init__(self):
        self.all_skus = {}
        self.model = DeepModel()
        self.predict_time = 0
        self.time_search = 0
        self.count_frame = 0
        self.top_k = 5

    def extract_features_from_img(self, cur_img):
        """Судя по названию эта функция извлекает фичи из тестового изображения"""
        cur_img = cv2.resize(cur_img, (224, 224))
        img = preprocess_input(cur_img)
        img = np.expand_dims(img, axis=0)
        feature = self.model.extract_feature(img)
        return feature

    def predict(self, img):
        self.count_frame += 1
        before_time = time.time()
        target_features = self.extract_features_from_img(img)
        self.predict_time += time.time() - before_time
        max_distance = 0
        result_dish = 0

        for dish, features_all in self.all_skus.items():
            for features in features_all:
                cur_distance = self.model.cosine_distance(target_features, features)
                cur_distance = cur_distance[0][0]
                if cur_distance > max_distance:
                    max_distance = cur_distance
                    result_dish = dish

        return result_dish, max_distance

    def add_img(self, img_path, id_img):
        img = cv2.imread(img_path)
        cur_img = img
        feature = self.extract_features_from_img(cur_img)
        if id_img not in self.all_skus:
            self.all_skus[id_img] = []
        self.all_skus[id_img].append(feature)
        return feature

    def remove_by_id(self, id_img):
        if id_img in self.all_skus:
            self.all_skus.pop(id_img)

    def remove_all(self):
        self.all_skus.clear()

    def add_img_from_pickle(self, id_img, pickle_path):
        res = pickle.load(open(pickle_path, 'rb'))
        self.all_skus[id_img] = res

    def get_additional_info(self):
        json_res = {}
        json_res["Extract features, time"] = self.predict_time
        json_res["Find nearest, time"] = self.time_search
        json_res["Count frame"] = self.count_frame
        json_res["RPS"] = self.count_frame / (self.predict_time + self.time_search)
        return json_res

In [3]:
if __name__ == "__main__":
    """тут немножко парсим аргументы изображения путь которому передаётся через командную строку. Создаётся объект,
    устанавливаются флаги для передачи изображения, создаётся переменная с аргументами, читается изображение 
    и аргумент image кладётся в переменную с таким же названием."""
#     ap = argparse.ArgumentParser()
#     ap.add_argument("-i", "--image", required=True, help="path to input image")
#     args = vars(ap.parse_args())

#     image = cv2.imread(args["image"])
    
    """загрузим и обработаем входящее изображение и изображения с которыми сравниваем"""
#     image = cv2.imread('Odata/red_h.jpg')
    inp_classifaier = ImageClassifier()
    d_imgs = "Odata"
    d_path = os.listdir(d_imgs)
    
    for f in d_path:
        inp_classifaier.add_img(os.path.join(d_imgs, f), f)
    
    inp_classifaier_all_skus_list = list(inp_classifaier.__dict__['all_skus'].keys())
    test1 = np.array(inp_classifaier.__dict__['all_skus'][str(inp_classifaier_all_skus_list[0])][0])
    np.savetxt("Odata/test1.csv", test1, delimiter=',')
    
    
    classifier = ImageClassifier()
    t_imgs = "Otest"
    t_paths = os.listdir(t_imgs)
    
    for f in t_paths:
        classifier.add_img(os.path.join(t_imgs, f), f)
    
    classifaier_all_skus_list = list(classifier.__dict__['all_skus'].keys())
    test2 = np.array(classifier.__dict__['all_skus'][str(classifaier_all_skus_list[0])][0])
    np.savetxt("Otest/test2.csv", test2, delimiter=',')
    
    print(inp_classifaier_all_skus_list, '/n')
    print(classifaier_all_skus_list)
    
    
#     """Создаём объект сравнения"""
#     similarity = ImageSimilarity()
    
#     '''Setup'''
#     similarity.batch_size = 16
#     similarity.num_processes = 2
    
#     '''Load source data'''
#     test1_sim = similarity.load_data_csv('Odata/test1.csv', delimiter=',')
#     test2_sim = similarity.load_data_csv('Otest/test2.csv', delimiter=',')#, cols=['id', 'url'])

#     '''Save features and fields'''
#     similarity.save_data('test1', test1)
#     similarity.save_data('test2', test2)

#     '''Calculate similarities'''
#     result = similarity.iteration(['test1_id', 'test1_url', 'test2_id', 'test2_url'], thresh=0.845)
#     print('Row for source file 1, and column for source file 2.')
#     print(result)
        
#     print(classifier.__dict__)
#     print(in_classifaier.__dict__)
#     cv2.imshow("Image", image)
#     cv2.waitKey(0)


Loading MobileNet.

Loading MobileNet.



In [4]:
"""Создаём объект сравнения"""
similarity = ImageSimilarity()

'''Setup'''
similarity.batch_size = 16
similarity.num_processes = 2

'''Load source data'''
test1_sim = similarity.load_data_csv('Odata/test1.csv', delimiter=',')
test2_sim = similarity.load_data_csv('Otest/test2.csv', delimiter=',')#, cols=['id', 'url'])

'''Save features and fields'''
similarity.save_data('test1', test1_sim)
similarity.save_data('test2', test2_sim)

'''Calculate similarities'''
result = similarity.iteration(['test1_id', 'test1_url', 'test2_id', 'test2_url'], thresh=0.845)
print('Row for source file 1, and column for source file 2.')
print(result)

C:\Users\omelchenko\Documents\GitHub\image_classifaier\multi.py:144: UserWarning: genfromtxt: Empty input file: "Odata/test1.csv"
  data = np.genfromtxt(
C:\Users\omelchenko\Documents\GitHub\image_classifaier\multi.py:144: UserWarning: genfromtxt: Empty input file: "Otest/test2.csv"
  data = np.genfromtxt(


Loading MobileNet.

test1: download starts.


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
inp_classifaier.__dict__

In [ ]:
inp_classifaier.__dict__['all_skus'][str(inp_classifaier_all_skus_list[0])][0]#['t_img.jpg'][0]

In [ ]:
inp_classifaier_all_skus_list = list(inp_classifaier.__dict__['all_skus'].keys())
print(inp_classifaier_all_skus_list)

In [ ]:
for k, v in classifier.__dict__['all_skus'].items():
    print(k, '------>', v)

In [ ]:
for i classifier.__dict__['all_skus']['santahat.jpg']

In [ ]:
inp_classifaier.__dict__['all_skus']['t_img.jpg']